#Tensorflow Models

In [ ]:
#code for all tensorflow models
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50, VGG16, InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

def plot_accuracy_loss(history):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.show()

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.show()

# Set hyperparameters
batch_size = 16
PATH = "../data/CUB_200_2011"
labels = pd.read_csv(os.path.join(PATH, "image_class_labels.txt"), sep=" ", header=None, names=["img_id", "label"])
images = pd.read_csv(os.path.join(PATH, "images.txt"), sep=" ", header=None, names=["img_id", "filepath"])
classes = pd.read_csv(os.path.join(PATH, "classes.txt"), sep=" ", header=None, names=["label", "category"])
num_classes = len(classes)
df = pd.merge(images, labels, on="img_id")
df = pd.merge(df, classes, on="label")
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])

# Initialize transformations for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5], # Mimics ColorJitter's brightness adjustment
    channel_shift_range=0.2, # Partially mimics ColorJitter's hue adjustment
    fill_mode='nearest'
)

# Load the dataset and create generators
train_generator = train_datagen.flow_from_dataframe(
    directory=os.path.join(PATH, "images"),
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    dataframe=train_df,
    x_col="filepath",
    y_col="category",
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5], # Mimics ColorJitter's brightness adjustment
    channel_shift_range=0.2, # Partially mimics ColorJitter's hue adjustment
    fill_mode='nearest'
)

test_generator = test_datagen.flow_from_dataframe(
    directory=os.path.join(PATH, "images"),
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    dataframe=test_df,
    x_col="filepath",
    y_col="category",
)
val_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5], # Mimics ColorJitter's brightness adjustment
    channel_shift_range=0.2, # Partially mimics ColorJitter's hue adjustment
    fill_mode='nearest'
)

val_generator = val_datagen.flow_from_dataframe(
    directory=os.path.join(PATH, "images"),
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    dataframe=val_df,
    x_col="filepath",
    y_col="category",
)

In [ ]:
#VGG16 early stopping
early_stopping = EarlyStopping(patience=20, restore_best_weights=True, min_delta=0, monitor='val_accuracy')
num_epochs = 200
leanring_rate = 0.0001

vgg16Model = VGG16(weights='imagenet', include_top=False)
x = GlobalAveragePooling2D()(vgg16Model.output)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=vgg16Model.input, outputs=predictions)

model.compile(optimizer=Adam(lr=leanring_rate), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_generator, epochs=num_epochs, validation_data=val_generator ,callbacks=[early_stopping])
plot_accuracy_loss(history)

model.save('../models/vgg16_EARLY.h5')
model = tf.keras.models.load_model('../models/vgg16_EARLY.h5')
model.evaluate(test_generator)

In [ ]:
#VGG16 200 epochs
num_epochs = 200
leanring_rate = 0.0001

vgg16Model = VGG16(weights='imagenet', include_top=False)
x = GlobalAveragePooling2D()(vgg16Model.output)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=vgg16Model.input, outputs=predictions)

model.compile(optimizer=Adam(lr=leanring_rate), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_generator, epochs=num_epochs, validation_data=val_generator)
plot_accuracy_loss(history)

model.save('../models/vgg16.h5')
model = tf.keras.models.load_model('../models/vgg16.h5')
model.evaluate(test_generator)

In [ ]:
#Resnet 50 early stopping
early_stopping = EarlyStopping(patience=20, restore_best_weights=True, min_delta=0, monitor='val_accuracy')
num_epochs = 200
learning_rate = 0.0001
# Define the model
resnetModel = ResNet50(weights='imagenet', include_top=False)
x = GlobalAveragePooling2D()(resnetModel.output)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=resnetModel.input, outputs=predictions)
# Compile the model
model.compile(optimizer=Adam(lr=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model
history = model.fit(train_generator, epochs=num_epochs, validation_data=val_generator,callbacks=[early_stopping])
plot_accuracy_loss(history)

model.save('../models/ResNet50_EARLY.h5')
model = tf.keras.models.load_model('../models/ResNet50_EARLY.h5')
model.evaluate(test_generator)

In [ ]:
#Resnet 50 200 epochs
num_epochs = 200
learning_rate = 0.0001
# Define the model
resnetModel = ResNet50(weights='imagenet', include_top=False)
x = GlobalAveragePooling2D()(resnetModel.output)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=resnetModel.input, outputs=predictions)
# Compile the model
model.compile(optimizer=Adam(lr=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model
history = model.fit(train_generator, epochs=num_epochs, validation_data=val_generator)
plot_accuracy_loss(history)

model.save('../models/ResNet50.h5')
model = tf.keras.models.load_model('../models/ResNet50.h5')
model.evaluate(test_generator)

In [ ]:
#generators for 299x299 images
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5], # Mimics ColorJitter's brightness adjustment
    channel_shift_range=0.2, # Partially mimics ColorJitter's hue adjustment
    fill_mode='nearest'
)

# Load the dataset and create generators
train_generator = train_datagen.flow_from_dataframe(
    directory=os.path.join(PATH, "images"),
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode='categorical',
    dataframe=train_df,
    x_col="filepath",
    y_col="category",
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5], # Mimics ColorJitter's brightness adjustment
    channel_shift_range=0.2, # Partially mimics ColorJitter's hue adjustment
    fill_mode='nearest'
)

test_generator = test_datagen.flow_from_dataframe(
    directory=os.path.join(PATH, "images"),
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    dataframe=test_df,
    x_col="filepath",
    y_col="category",
)
val_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5], # Mimics ColorJitter's brightness adjustment
    channel_shift_range=0.2, # Partially mimics ColorJitter's hue adjustment
    fill_mode='nearest'
)

val_generator = val_datagen.flow_from_dataframe(
    directory=os.path.join(PATH, "images"),
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    dataframe=val_df,
    x_col="filepath",
    y_col="category",
)

In [ ]:
#InceptionV3 early stopping
early_stopping = EarlyStopping(patience=20, restore_best_weights=True, min_delta=0, monitor='val_accuracy')
num_epochs = 200
learning_rate = 0.0001
# Define the model
inceptionModel = InceptionV3(weights='imagenet', include_top=False)
x = GlobalAveragePooling2D()(inceptionModel.output)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=inceptionModel.input, outputs=predictions)
model.compile(optimizer=Adam(lr=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model
history = model.fit(train_generator, epochs=num_epochs,validation_data=val_generator, callbacks=[early_stopping])
plot_accuracy_loss(history)
model.save('../models/InceptionV3_EARLY.h5')

model = tf.keras.models.load_model('../models/InceptionV3_EARLY.h5')
model.evaluate(test_generator)

In [ ]:
#incepionV3 200 epochs
num_epochs = 200
learning_rate = 0.0001
# Define the model
inceptionModel = InceptionV3(weights='imagenet', include_top=False)
x = GlobalAveragePooling2D()(inceptionModel.output)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=inceptionModel.input, outputs=predictions)
model.compile(optimizer=Adam(lr=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model
history = model.fit(train_generator, epochs=num_epochs,validation_data=val_generator)
plot_accuracy_loss(history)
model.save('../models/InceptionV3.h5')

model = tf.keras.models.load_model('../models/InceptionV3.h5')
model.evaluate(test_generator)

#Pytorch models

In [ ]:
#code for all pytorch models
import os
import time
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import tqdm
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler


run_on_local = 1

if run_on_local == 1:
    working_directory = r'C:\Users\ODIN\Desktop\ml_final'
    os.chdir(working_directory)
    print("Current Working Directory: ", os.getcwd())
else:
    from google.colab import drive
    drive.mount('/content/drive')
    !mkdir /content/data
    !tar -xvzf /content/drive/MyDrive/ML_final/CUB_200_2011.tgz -C /content/data

# if run_on_local == 1:
#     %cd C:\Users\ODIN\Desktop\ml_final\

# Create Training/Val Loss Graphs
def create_graph(epoch_train_loss, epoch_val_loss):
    plt.plot(epoch_train_loss, label='Training Loss')
    plt.plot(epoch_val_loss, label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    return plt.show()

# Early Stopping
# https://www.youtube.com/watch?v=lMMlbmfvKDQ
# https://github.com/jeffheaton/app_deep_learning/blob/main/t81_558_class_03_4_early_stop.ipynb

import copy

class EarlyStopping:
    def __init__(self, patience=5, min_delta=0, restore_best_weights=True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_model = None
        self.best_loss = None
        self.counter = 0
        self.status = ""

    def __call__(self, model, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
        elif self.best_loss - val_loss >= self.min_delta:
            self.best_model = copy.deepcopy(model.state_dict())
            self.best_loss = val_loss
            self.counter = 0
            self.status = f"Improvement found, counter reset to {self.counter}"
        else:
            self.counter += 1
            self.status = f"No improvement in the last {self.counter} epochs"
            if self.counter >= self.patience:
                self.status = f"Early stopping triggered after {self.counter} epochs."
                if self.restore_best_weights:
                    model.load_state_dict(self.best_model)
                return True
        return False
    
#check aviable weights
import torch

weight_enum = torch.hub.load("pytorch/vision", "get_model_weights", name="inception_v3")
print([weight for weight in weight_enum])


In [ ]:
#VGG16
transform = transforms.Compose([
    transforms.Resize((224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=45),
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


#dataset = torchvision.datasets.ImageFolder(root='/content/data/CUB_200_2011/images', transform=transform
dataset = torchvision.datasets.ImageFolder(root='./content/data/CUB_200_2011/images', transform=transform)
# Split to 80%/10%/10%

train_split = int(len(dataset) * .8)
val_split = int(len(dataset) * .1)
test_split = int(len(dataset) * .1)

#Make sure splits match 1:1 with dataset
total = train_split + val_split + test_split
if total < len(dataset):
    train_split = train_split + (len(dataset) - total)



print(dataset, train_split, val_split, test_split, train_split + val_split + test_split )
train, val, test = torch.utils.data.random_split(dataset, [train_split, val_split, test_split])

num_workers=2
batch_size=16

trainLoader = DataLoader(train , batch_size=batch_size,
                                           num_workers=num_workers,  shuffle=True)
valLoader = DataLoader(val, batch_size=batch_size,
                                          num_workers=num_workers )
testLoader = DataLoader(test, batch_size=batch_size,
                                          num_workers=num_workers)

import torchvision.models as models

torch.cuda.empty_cache()

weightz = models.get_model_weights("vgg16")
weightz = weightz.DEFAULT
print(weightz)

vgg16 = models.vgg16(weights=weightz)

num_classes = len(dataset.classes)
print("classes:", num_classes)

vgg16.classifier[6] = torch.nn.Linear(vgg16.classifier[6].in_features, num_classes)

# Create an object of the model
vgg16 = vgg16.to("cuda")

# optimizer
optimizer = optim.RMSprop(vgg16.parameters(), lr=1e-4)

# Loss
loss_function = nn.CrossEntropyLoss()


import tqdm

es = EarlyStopping(patience=20, min_delta=0, restore_best_weights=True)


epoch_train_loss = []
epoch_val_loss = []

num_epochs = 200
for epoch in range(num_epochs):
    # Train
    train_loss = []
    vgg16.train()
    pbar = tqdm.tqdm(trainLoader, total=len(trainLoader))
    for images, labels in pbar:
        images = images.to("cuda")
        labels = labels.to("cuda")

        optimizer.zero_grad()
        outputs = vgg16(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss.append(loss.item())
        pbar.set_description(f"Epoch: [{epoch + 1}/{num_epochs}], Train Loss: {loss.item():.4f}")

    avg_train_loss = np.mean(train_loss)
    epoch_train_loss.append(avg_train_loss)

    # Validation
    vgg16.eval()
    val_loss = []
    with torch.no_grad():
        for images, labels in valLoader:
            images = images.to("cuda")
            labels = labels.to("cuda")
            outputs = vgg16(images)
            loss = loss_function(outputs, labels)
            val_loss.append(loss.item())

    avg_val_loss = np.mean(val_loss)
    epoch_val_loss.append(avg_val_loss)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Val Loss: {np.mean(avg_val_loss):.4f}')

    # Check for ES
    if es(vgg16, avg_val_loss):
        pbar.set_description(f"Epoch: {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, {es.status}")
        break
    else:
        pbar.set_description(f"Epoch: {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

print("Training complete.")
create_graph(epoch_train_loss, epoch_val_loss)
#Test the model
y_test = []
y_test_predict = []

vgg16.eval()
for images, labels in testLoader:
    images = images.to("cuda")
    labels = labels.to("cuda")

    with torch.no_grad():
        outputs = vgg16(images).cpu()
        _, predicted = torch.max(outputs, 1)
        y_test_predict.extend(predicted.numpy())
        y_test.extend(labels.cpu().numpy())

print("Test Accuracy: ", accuracy_score(y_test, y_test_predict))

if run_on_local == 1:
    save_path = './'
else:
    save_path = '/content/drive/MyDrive/ML_final/'

torch.save({
            'epoch': epoch,
            'model_state_dict': vgg16.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
            }, save_path + '200_vgg16.pth')

In [ ]:
#Resnet50
transform = transforms.Compose([
    transforms.Resize((224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=45),
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


#dataset = torchvision.datasets.ImageFolder(root='/content/data/CUB_200_2011/images', transform=transform
dataset = torchvision.datasets.ImageFolder(root='./content/data/CUB_200_2011/images', transform=transform)
# Split to 80%/10%/10%

train_split = int(len(dataset) * .8)
val_split = int(len(dataset) * .1)
test_split = int(len(dataset) * .1)

#Make sure splits match 1:1 with dataset
total = train_split + val_split + test_split
if total < len(dataset):
    train_split = train_split + (len(dataset) - total)



print(dataset, train_split, val_split, test_split, train_split + val_split + test_split )
train, val, test = torch.utils.data.random_split(dataset, [train_split, val_split, test_split])

num_workers=2
batch_size=16

trainLoader = DataLoader(train , batch_size=batch_size,
                                           num_workers=num_workers,  shuffle=True)
valLoader = DataLoader(val, batch_size=batch_size,
                                          num_workers=num_workers )
testLoader = DataLoader(test, batch_size=batch_size,
                                          num_workers=num_workers)
import torchvision.models as models


torch.cuda.empty_cache()

weightz = models.get_model_weights("resnet50")
weightz = weightz.DEFAULT
print(weightz)

resnet50 = models.resnet50(weights=weightz)

num_classes = len(dataset.classes)
print("classes:", num_classes)

for param in resnet50.parameters():
    param.requires_grad = False

num_features = resnet50.fc.in_features
resnet50.fc = torch.nn.Linear(num_features, num_classes)

# Create an object of the model
resnet50 = resnet50.to("cuda")

# optimizer
optimizer = optim.RMSprop(resnet50.parameters(), lr=1e-4)

# Loss
loss_function = nn.CrossEntropyLoss()
import tqdm

es = EarlyStopping(patience=20, min_delta=0, restore_best_weights=True)


epoch_train_loss = []
epoch_val_loss = []

num_epochs = 200
for epoch in range(num_epochs):
    # Train
    train_loss = []
    resnet50.train()
    pbar = tqdm.tqdm(trainLoader, total=len(trainLoader))
    for images, labels in pbar:
        images = images.to("cuda")
        labels = labels.to("cuda")

        optimizer.zero_grad()
        outputs = resnet50(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss.append(loss.item())
        pbar.set_description(f"Epoch: [{epoch + 1}/{num_epochs}], Train Loss: {loss.item():.4f}")

    avg_train_loss = np.mean(train_loss)
    epoch_train_loss.append(avg_train_loss)

    # Validation
    resnet50.eval()
    val_loss = []
    with torch.no_grad():
        for images, labels in valLoader:
            images = images.to("cuda")
            labels = labels.to("cuda")
            outputs = resnet50(images)
            loss = loss_function(outputs, labels)
            val_loss.append(loss.item())

    avg_val_loss = np.mean(val_loss)
    epoch_val_loss.append(avg_val_loss)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Val Loss: {np.mean(avg_val_loss):.4f}')

    # Check for ES
    if es(resnet50, avg_val_loss):
        pbar.set_description(f"Epoch: {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, {es.status}")
        break
    else:
        pbar.set_description(f"Epoch: {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

print("Training complete.")
create_graph(epoch_train_loss, epoch_val_loss)
import tqdm

es = EarlyStopping(patience=20, min_delta=0, restore_best_weights=True)


epoch_train_loss = []
epoch_val_loss = []

num_epochs = 200
for epoch in range(num_epochs):
    # Train
    train_loss = []
    resnet50.train()
    pbar = tqdm.tqdm(trainLoader, total=len(trainLoader))
    for images, labels in pbar:
        images = images.to("cuda")
        labels = labels.to("cuda")

        optimizer.zero_grad()
        outputs = resnet50(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss.append(loss.item())
        pbar.set_description(f"Epoch: [{epoch + 1}/{num_epochs}], Train Loss: {loss.item():.4f}")

    avg_train_loss = np.mean(train_loss)
    epoch_train_loss.append(avg_train_loss)

    # Validation
    resnet50.eval()
    val_loss = []
    with torch.no_grad():
        for images, labels in valLoader:
            images = images.to("cuda")
            labels = labels.to("cuda")
            outputs = resnet50(images)
            loss = loss_function(outputs, labels)
            val_loss.append(loss.item())

    avg_val_loss = np.mean(val_loss)
    epoch_val_loss.append(avg_val_loss)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Val Loss: {np.mean(avg_val_loss):.4f}')

    # Check for ES
    if es(resnet50, avg_val_loss):
        pbar.set_description(f"Epoch: {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, {es.status}")
        break
    else:
        pbar.set_description(f"Epoch: {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

print("Training complete.")
create_graph(epoch_train_loss, epoch_val_loss)

if run_on_local == 1:
    save_path = './'
else:
    save_path = '/content/drive/MyDrive/ML_final/'

torch.save({
            'epoch': epoch,
            'model_state_dict': resnet50.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
            }, save_path + '200_resnet50.pth')


In [ ]:
#InceptionV3
transform = transforms.Compose([
    transforms.Resize((299)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=45),
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


#dataset = torchvision.datasets.ImageFolder(root='/content/data/CUB_200_2011/images', transform=transform
dataset = torchvision.datasets.ImageFolder(root='./content/data/CUB_200_2011/images', transform=transform)
# Split to 80%/10%/10%

train_split = int(len(dataset) * .8)
val_split = int(len(dataset) * .1)
test_split = int(len(dataset) * .1)

#Make sure splits match 1:1 with dataset
total = train_split + val_split + test_split
if total < len(dataset):
    train_split = train_split + (len(dataset) - total)



print(dataset, train_split, val_split, test_split, train_split + val_split + test_split )
train, val, test = torch.utils.data.random_split(dataset, [train_split, val_split, test_split])

num_workers=2
batch_size=16

trainLoader = DataLoader(train , batch_size=batch_size,
                                           num_workers=num_workers,  shuffle=True)
valLoader = DataLoader(val, batch_size=batch_size,
                                          num_workers=num_workers )
testLoader = DataLoader(test, batch_size=batch_size,
                                          num_workers=num_workers)

import torchvision.models as models
from torch.optim import lr_scheduler

torch.cuda.empty_cache()

weightz = models.get_model_weights("inception_v3")
weightz = weightz.DEFAULT
print(weightz)

inception = models.inception_v3(weights=weightz)

for param in inception.parameters():
    param.requires_grad = False

num_classes = len(dataset.classes)
print("classes:", num_classes)

num_features = inception.AuxLogits.fc.in_features
inception.AuxLogits.fc = torch.nn.Linear(num_features, num_classes)

num_features = inception.fc.in_features
inception.fc = torch.nn.Linear(num_features, num_classes)



params_to_update = []
for param in inception.fc.parameters():
    param.requires_grad = True

# Create an object of the model
inception = inception.to("cuda")


# optimizer
optimizer = optim.RMSprop(inception.parameters(), lr=1e-4)

# Loss
loss_function = nn.CrossEntropyLoss()

# scheduler
scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

import tqdm

es = EarlyStopping(patience=20, min_delta=0, restore_best_weights=True)

epoch_train_loss = []
epoch_val_loss = []

num_epochs = 200
for epoch in range(num_epochs):
    # Train
    train_loss = []
    #scheduler.step()
    inception.train()
    pbar = tqdm.tqdm(trainLoader, total=len(trainLoader))
    for images, labels in pbar:
        images = images.to("cuda")
        labels = labels.to("cuda")

        optimizer.zero_grad()

        outputs, aux_outputs = inception(images)
        loss1 = loss_function(outputs, labels)
        loss2 = loss_function(aux_outputs, labels)
        loss = loss1 + 0.4*loss2

        # loss = loss_function(outputs, labels)

        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())
        pbar.set_description(f"Epoch: [{epoch + 1}/{num_epochs}], Train Loss: {loss.item():.4f}")

    avg_train_loss = np.mean(train_loss)
    epoch_train_loss.append(avg_train_loss)

    # Validation
    inception.eval()
    val_loss = []
    with torch.no_grad():
        for images, labels in valLoader:
            images = images.to("cuda")
            labels = labels.to("cuda")
            outputs = inception(images)
            loss = loss_function(outputs, labels)
            val_loss.append(loss.item())

    avg_val_loss = np.mean(val_loss)
    epoch_val_loss.append(avg_val_loss)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Val Loss: {np.mean(avg_val_loss):.4f}')

    # Check for ES
    if es(inception, avg_val_loss):
        pbar.set_description(f"Epoch: {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, {es.status}")
        break
    else:
        pbar.set_description(f"Epoch: {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

print("Training complete.")
create_graph(epoch_train_loss, epoch_val_loss)

#Test the model
y_test = []
y_test_predict = []

inception.eval()
for images, labels in testLoader:
    images = images.to("cuda")
    labels = labels.to("cuda")

    with torch.no_grad():
        outputs = inception(images).cpu()
        _, predicted = torch.max(outputs, 1)
        y_test_predict.extend(predicted.numpy())
        y_test.extend(labels.cpu().numpy())

print("Test Accuracy: ", accuracy_score(y_test, y_test_predict))

if run_on_local == 1:
    save_path = './'
else:
    save_path = '/content/drive/MyDrive/ML_final/'

torch.save({
            'epoch': epoch,
            'model_state_dict': inception.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
            }, save_path + '200_inception.pth')